In [2]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import os # accessing directory structure
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
print(os.listdir('.'))

['.ipynb_checkpoints', 'Anti_fraud_analyst_q1.ipynb', 'Anti_fraud_code', 'Anti_fraud_detection_q3.ipynb', 'creditcard', 'creditcard.csv.zip', 'ieee-fraud-detection', 'ieee-fraud-detection.zip', 'path', 'paysim_profile.html', 'PS_20174392719_1491204439457_log.csv', 'Untitled1.ipynb']


In [4]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

df = pd.read_csv('PS_20174392719_1491204439457_log.csv')
df = df.rename(columns={'oldbalanceOrg':'oldBalanceOrig', 'newbalanceOrig':'newBalanceOrig', \
                        'oldbalanceDest':'oldBalanceDest', 'newbalanceDest':'newBalanceDest'})
print(df.head())

#test for missing values 
df.isnull().values.any()

#drop duplicates
df = df.drop_duplicates()

   step      type    amount     nameOrig  oldBalanceOrig  newBalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815        170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295         21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145           181.0            0.00   
3     1  CASH_OUT    181.00   C840083671           181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720         41554.0        29885.86   

      nameDest  oldBalanceDest  newBalanceDest  isFraud  isFlaggedFraud  
0  M1979787155             0.0             0.0        0               0  
1  M2044282225             0.0             0.0        0               0  
2   C553264065             0.0             0.0        1               0  
3    C38997010         21182.0             0.0        1               0  
4  M1230701703             0.0             0.0        0               0  


In [6]:

## DATA CLEANING
#After reviewing the data we noted that fraud only occurs in 'TRANSFER's and 'CASH_OUT's. So we assemble only the corresponding data in X for analysis.

X = df.loc[(df.type == 'TRANSFER') | (df.type == 'CASH_OUT')]

randomState = 5
np.random.seed(randomState)

#X = X.loc[np.random.choice(X.index, 100000, replace = False)]

Y = X['isFraud']
del X['isFraud']

# Eliminate columns shown to be irrelevant for analysis in the EDA
X = X.drop(['nameOrig', 'nameDest', 'isFlaggedFraud'], axis = 1)

# Binary-encoding of labelled data in 'type'
X.loc[X.type == 'TRANSFER', 'type'] = 0
X.loc[X.type == 'CASH_OUT', 'type'] = 1
X.type = X.type.astype(int) # convert dtype('O') to dtype(int)

#3.1. Imputation of Latent Missing Values
#The data has several transactions with zero balances in the destination account both before and after a non-zero amount is transacted. The fraction of such transactions, where zero likely denotes a missing value, is much larger in fraudulent (50%) compared to genuine transactions (0.06%).

In [26]:
Xfraud = X.loc[Y == 1]
XnonFraud = X.loc[Y == 0]
print('\nThe fraction of fraudulent transactions with \'oldBalanceDest\' = \
\'newBalanceDest\' = 0 although the transacted \'amount\' is non-zero is: {}'.\
format(len(Xfraud.loc[(Xfraud.oldBalanceDest == 0) & \
(Xfraud.newBalanceDest == 0) & (Xfraud.amount)]) / (1.0 * len(Xfraud))))

print('\nThe fraction of genuine transactions with \'oldBalanceDest\' = \
newBalanceDest\' = 0 although the transacted \'amount\' is non-zero is: {}'.\
format(len(XnonFraud.loc[(XnonFraud.oldBalanceDest == 0) & \
(XnonFraud.newBalanceDest == 0) & (XnonFraud.amount)]) / (1.0 * len(XnonFraud))))


The fraction of fraudulent transactions with 'oldBalanceDest' = 'newBalanceDest' = 0 although the transacted 'amount' is non-zero is: 0.0

The fraction of genuine transactions with 'oldBalanceDest' = newBalanceDest' = 0 although the transacted 'amount' is non-zero is: 0.0


In [28]:
## look at the fraudulent transactions
print('\n The types of fraudulent transactions are {}'.format(\
list(df.loc[df.isFraud == 1].type.drop_duplicates().values))) # only 'CASH_OUT' 
                                                             # & 'TRANSFER'

dfFraudTransfer = df.loc[(df.isFraud == 1) & (df.type == 'TRANSFER')]
dfFraudCashout = df.loc[(df.isFraud == 1) & (df.type == 'CASH_OUT')]

print ('\n The number of fraudulent TRANSFERs = {}'.\
       format(len(dfFraudTransfer))) # 4097

print ('\n The number of fraudulent CASH_OUTs = {}'.\
       format(len(dfFraudCashout))) # 4116


 The types of fraudulent transactions are ['TRANSFER', 'CASH_OUT']

 The number of fraudulent TRANSFERs = 4097

 The number of fraudulent CASH_OUTs = 4116


In [30]:
## see if column is fraudulant is relevant 
print('\nThe type of transactions in which isFlaggedFraud is set: \
{}'.format(list(df.loc[df.isFlaggedFraud == 1].type.drop_duplicates()))) 
                                                            # only 'TRANSFER'

dfTransfer = df.loc[df.type == 'TRANSFER']
dfFlagged = df.loc[df.isFlaggedFraud == 1]
dfNotFlagged = df.loc[df.isFlaggedFraud == 0]

print('\nMin amount transacted when isFlaggedFraud is set= {}'\
                                  .format(dfFlagged.amount.min())) # 353874.22

print('\nMax amount transacted in a TRANSFER where isFlaggedFraud is not set=\
 {}'.format(dfTransfer.loc[dfTransfer.isFlaggedFraud == 0].amount.max())) # 92445516.64


The type of transactions in which isFlaggedFraud is set: ['TRANSFER']

Min amount transacted when isFlaggedFraud is set= 353874.22

Max amount transacted in a TRANSFER where isFlaggedFraud is not set= 92445516.64


In [32]:
#Imputation of Latent Missing Values¶
#The data has several transactions with zero balances in the destination account both before and after a non-zero amount is transacted. The fraction of such transactions, where zero likely denotes a missing value, is much larger in fraudulent (50%) compared to genuine transactions (0.06%).
Xfraud = X.loc[Y == 1]
XnonFraud = X.loc[Y == 0]
print('\nThe fraction of fraudulent transactions with \'oldBalanceDest\' = \
\'newBalanceDest\' = 0 although the transacted \'amount\' is non-zero is: {}'.\
format(len(Xfraud.loc[(Xfraud.oldBalanceDest == 0) & \
(Xfraud.newBalanceDest == 0) & (Xfraud.amount)]) / (1.0 * len(Xfraud))))

print('\nThe fraction of genuine transactions with \'oldBalanceDest\' = \
newBalanceDest\' = 0 although the transacted \'amount\' is non-zero is: {}'.\
format(len(XnonFraud.loc[(XnonFraud.oldBalanceDest == 0) & \
(XnonFraud.newBalanceDest == 0) & (XnonFraud.amount)]) / (1.0 * len(XnonFraud))))


The fraction of fraudulent transactions with 'oldBalanceDest' = 'newBalanceDest' = 0 although the transacted 'amount' is non-zero is: 0.0

The fraction of genuine transactions with 'oldBalanceDest' = newBalanceDest' = 0 although the transacted 'amount' is non-zero is: 0.0


In [34]:
#Since the destination account balances being zero is a strong indicator of fraud, we do not impute the account balance (before the transaction is made) with a statistic or from a distribution with a subsequent adjustment for the amount transacted. Doing so would mask this indicator of fraud and make fraudulent transactions appear genuine. Instead, below we replace the value of 0 with -1 which will be more useful to a suitable machine-learning (ML) algorithm detecting fraud.
X.loc[(X.oldBalanceDest == 0) & (X.newBalanceDest == 0) & (X.amount != 0), \
      ['oldBalanceDest', 'newBalanceDest']] = - 1

In [36]:
#The data also has several transactions with zero balances in the originating account both before and after a non-zero amount is transacted. In this case, the fraction of such transactions is much smaller in fraudulent (0.3%) compared to genuine transactions (47%). Once again, from similar reasoning as above, instead of imputing a numerical value we replace the value of 0 with a null value.
X.loc[(X.oldBalanceOrig == 0) & (X.newBalanceOrig == 0) & (X.amount != 0), \
      ['oldBalanceOrig', 'newBalanceOrig']] = np.nan

In [70]:
#4. Feature-engineering
#Motivated by the possibility of zero-balances serving to differentiate between fraudulent and genuine transactions, we take the data-imputation of section 3.1 a step further and create 2 new features (columns) recording errors in the originating and destination accounts for each transaction. These new features turn out to be important in obtaining the best performance from the ML algorithm that we will finally use.


X['errorBalanceOrig'] = X.newBalanceOrig + X.amount - X.oldBalanceOrig
X['errorBalanceDest'] = X.oldBalanceDest + X.amount - X.newBalanceDest



ValueError: Can only compare identically-labeled Series objects

In [54]:
Xfraud.head()

,step,type,amount,oldBalanceOrig,newBalanceOrig,oldBalanceDest,newBalanceDest,errorBalanceOrig,errorBalanceDest
2,1,0,181.0,181.0,0.0,-1.0,-1.0,0.0,181.0
3,1,1,181.0,181.0,0.0,21182.0,0.0,0.0,21363.0
251,1,0,2806.0,2806.0,0.0,-1.0,-1.0,0.0,2806.0
252,1,1,2806.0,2806.0,0.0,26202.0,0.0,0.0,29008.0
680,1,0,20128.0,20128.0,0.0,-1.0,-1.0,0.0,20128.0


In [42]:
#currency conversion 
#conversion_rate = 0.85  # Example: USD to EUR
#data['amount_converted'] = data['amount'] * conversion_rate


In [44]:
#amount is expressed in the local currency yet the data origin and location is not provided

In [56]:
#Timestamp Standardization:
#Convert timestamps into a readable format. We're choosing an arbitrary start date
df['step_datetime'] = pd.to_timedelta(df['step'], unit='h') + pd.Timestamp('2022-01-01')


In [58]:
print(df.head())

   step      type    amount     nameOrig  oldBalanceOrig  newBalanceOrig  \
0     1   PAYMENT   9839.64  C1231006815        170136.0       160296.36   
1     1   PAYMENT   1864.28  C1666544295         21249.0        19384.72   
2     1  TRANSFER    181.00  C1305486145           181.0            0.00   
3     1  CASH_OUT    181.00   C840083671           181.0            0.00   
4     1   PAYMENT  11668.14  C2048537720         41554.0        29885.86   

      nameDest  oldBalanceDest  newBalanceDest  isFraud  isFlaggedFraud  \
0  M1979787155             0.0             0.0        0               0   
1  M2044282225             0.0             0.0        0               0   
2   C553264065             0.0             0.0        1               0   
3    C38997010         21182.0             0.0        1               0   
4  M1230701703             0.0             0.0        0               0   

        step_datetime  
0 2022-01-01 01:00:00  
1 2022-01-01 01:00:00  
2 2022-01-01 01:00:0

In [60]:
#Feature Engineering: Create derived features, such as transaction type counts
df['isFraud_binary'] = df['isFraud'].apply(lambda x: 1 if x == 1 else 0)


In [20]:
#Step 4: Aggregation
#Aggregate data at different intervals (e.g., daily, hourly):
# Daily aggregation of total amounts and fraud cases
daily_summary = data.groupby(data['step_datetime'].dt.date).agg(
    total_transactions=('amount', 'count'),
    total_fraud_cases=('isFraud_binary', 'sum'),
    total_amount=('amount', 'sum')
).reset_index()


In [62]:
#Step 5: Data Quality Checks
#Null Values Check:
print(df.isnull().sum())


step              0
type              0
amount            0
nameOrig          0
oldBalanceOrig    0
newBalanceOrig    0
nameDest          0
oldBalanceDest    0
newBalanceDest    0
isFraud           0
isFlaggedFraud    0
step_datetime     0
isFraud_binary    0
dtype: int64


In [78]:
X.head()

,step,type,amount,oldBalanceOrig,newBalanceOrig,oldBalanceDest,newBalanceDest,errorBalanceOrig,errorBalanceDest,step_datetime
2,1,0,181.00,181.0,0.0,-1.0,-1.00,0.00,181.0,2022-01-01 01:00:00
3,1,1,181.00,181.0,0.0,21182.0,0.00,0.00,21363.0,2022-01-01 01:00:00
15,1,1,229133.94,15325.0,0.0,5083.0,51513.44,213808.94,182703.5,2022-01-01 01:00:00
19,1,0,215310.30,705.0,0.0,22425.0,0.00,214605.30,237735.3,2022-01-01 01:00:00
24,1,0,311685.89,10835.0,0.0,6267.0,2719172.89,300850.89,-2401220.0,2022-01-01 01:00:00


In [80]:
#Column Validation: Ensure balances align:
invalid_balances = X[X['oldBalanceOrig'] - data['amount'] != data['newBalanceOrig']]
print(f"Invalid rows: {len(invalid_balances)}")


NameError: name 'data' is not defined

In [84]:
#duplicate check
print(df.duplicated().sum())


0


In [85]:
#Step 6: Summary Statistics and Data Profiling
#Generate profiling reports for insights into the dataset:
#Summary Statistics:
print(df.describe())


               step        amount  oldBalanceOrig  newBalanceOrig  \
count  6.362620e+06  6.362620e+06    6.362620e+06    6.362620e+06   
mean   2.433972e+02  1.798619e+05    8.338831e+05    8.551137e+05   
min    1.000000e+00  0.000000e+00    0.000000e+00    0.000000e+00   
25%    1.560000e+02  1.338957e+04    0.000000e+00    0.000000e+00   
50%    2.390000e+02  7.487194e+04    1.420800e+04    0.000000e+00   
75%    3.350000e+02  2.087215e+05    1.073152e+05    1.442584e+05   
max    7.430000e+02  9.244552e+07    5.958504e+07    4.958504e+07   
std    1.423320e+02  6.038582e+05    2.888243e+06    2.924049e+06   

       oldBalanceDest  newBalanceDest       isFraud  isFlaggedFraud  \
count    6.362620e+06    6.362620e+06  6.362620e+06    6.362620e+06   
mean     1.100702e+06    1.224996e+06  1.290820e-03    2.514687e-06   
min      0.000000e+00    0.000000e+00  0.000000e+00    0.000000e+00   
25%      0.000000e+00    0.000000e+00  0.000000e+00    0.000000e+00   
50%      1.327057e+05  

In [88]:
#fraud rate
fraud_rate = df['isFraud'].mean() * 100
print(f"Fraud Rate: {fraud_rate:.2f}%")


Fraud Rate: 0.13%


In [19]:
pip install pandas-profiling

Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip install pandas-profiling

  Using cached pandas_profiling-3.2.0-py2.py3-none-any.whl (262 kB)
  Using cached joblib-1.1.1-py2.py3-none-any.whl (309 kB)
  Using cached htmlmin-0.1.12-py3-none-any.whl
  Using cached phik-0.12.0-cp36-cp36m-win_amd64.whl (660 kB)
  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached missingno-0.5.2-py3-none-any.whl (8.7 kB)
  Using cached pandas_profiling-3.1.0-py2.py3-none-any.whl (261 kB)
  Using cached requests-2.27.1-py2.py3-none-any.whl (63 kB)
  Using cached Jinja2-3.0.3-py3-none-any.whl (133 kB)
  Using cached joblib-1.0.1-py3-none-any.whl (303 kB)
  Using cached pydantic-1.9.2-cp36-cp36m-win_amd64.whl (2.1 MB)
  Using cached ImageHash-4.3.1-py2.py3-none-any.whl (296 kB)
  Using cached dataclasses-0.8-py3-none-any.whl (19 kB)
  Using cached charset_normalizer-2.0.12-py3-none-any.whl (39 kB)
  Using cached idna-3.10-py3-none-any.whl (70 kB)
  Using cached importlib_resources-5.4.0-py3-none-any.whl (28 kB)
  Attempting uninstall: joblib
    Found existing inst

In [ ]:
pip install --upgrade notebook

In [24]:
pip install --upgrade ipywidgets

Note: you may need to restart the kernel to use updated packages.


In [22]:
#Data Profiling with Pandas Profiling:

from pandas_profiling import ProfileReport

profile = ProfileReport(data, title="PaySim Data Profiling", explorative=True)
profile.to_file("paysim_profile.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

C:\Users\Lea\Anaconda3\envs\tensorflow_cpu\lib\site-packages\pandas\core\strings.py:1387: FutureWarning: split() requires a non-empty pattern match.
  f = lambda x: regex.split(x, maxsplit=n)


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]